<a href="https://colab.research.google.com/github/rpizarrog/innovacion-empresarial/blob/main/notebook_python/Caso_11_Validaci%C3%B3n_cruzada_para_regresi%C3%B3n_Datos_rendimiento_escolar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo

Realizar validación cruzada y evaluar modelos de regresión multivariada con datos de rendimiento escolar.





# Los datos

El contexto de los datos se relaciona con variables que afectan al rendimiento escolar de un alumno de educación superior.

## Variables independientes

* Horas de estudio por semana: Esta variable cuantifica el tiempo dedicado al estudio fuera del horario escolar. Un mayor número de horas de estudio se asocia generalmente con un mejor rendimiento escolar, aunque la calidad del estudio y la eficiencia también juegan un papel importante.

* Asistencia a clases (tasa de asistencia): Representa el porcentaje de clases a las que el alumno asiste durante un periodo determinado. Las altas tasas de asistencia suelen correlacionarse con un mejor rendimiento, ya que el estudiante tiene más oportunidades de aprender y participar en actividades educativas.

* Número de libros leídos por año: Esta variable mide el hábito de lectura del alumno fuera del material escolar obligatorio. Leer diversifica los conocimientos, mejora la comprensión lectora y fomenta habilidades analíticas, lo cual puede repercutir positivamente en el rendimiento escolar.

* Ingreso familiar: Medido como el ingreso total del hogar del alumno. Aunque el ingreso familiar no afecta directamente la capacidad cognitiva o el esfuerzo del estudiante, sí influye en la accesibilidad a recursos educativos, como libros, tecnología, clases particulares y un entorno de estudio adecuado, que pueden mejorar las condiciones de aprendizaje.

* Número de actividades extracurriculares: Cuantifica la participación del estudiante en actividades fuera del currículo académico, como deportes, música, arte, clubes de ciencia, entre otros. La participación en actividades extracurriculares puede fomentar habilidades sociales, de liderazgo y de gestión del tiempo, las cuales son útiles para el éxito académico. Sin embargo, una carga excesiva de actividades puede tener un efecto contrario, reduciendo el tiempo disponible para el estudio y la recuperación.

* Nivel de motivación. Valores numéricos que establece el grado de motivación de un estudiante.


## Variable dependite

* rendimiento escolar. Es el valor numérico del promedio escolar de un estudiante.



# Desarrollo

## Cargar librerías

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

import time # Para medir el tiempo de ejecucón

# Para árboles de regresión
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Para random Forest
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import DecisionTreeRegressor, plot_tree # Para visualiar árbol
from sklearn.tree import export_text # Para reglas de asociación


# Para modelo polinomial

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split # Para partir datos en datos entrenamieto y datos validación

# Para modelo de SVR
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split # Para partir datos en datos entrenamieto y datos validación

# Para escalar datos
from sklearn.preprocessing import StandardScaler # Para escalar datos

from sklearn import metrics # Para determinar métricas.
from sklearn.metrics import mean_squared_error, r2_score

# Para determinar estadísticos ...
import statsmodels.api as sm

# Para validación cruzada
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler

## Cargar funciones



In [2]:
def f_ver_temperaturas (fechas, temperaturas):
    # Crear un DataFrame con las fechas y temperaturas
    df = pd.DataFrame({'fecha': fechas, 'temperatura': temperaturas})

    # Asegurarse de que 'fecha' es un tipo datetime
    df['fecha'] = pd.to_datetime(df['fecha'])

    # Establecer 'fecha' como el índice del DataFrame
    df.set_index('fecha', inplace=True)

    # Agrupar por mes y año, y calcular la temperatura promedio
    df_resampled = df.resample('M').mean()

    # Crear la visualización
    plt.figure(figsize=(12, 6))
    plt.plot(df_resampled.index, df_resampled['temperatura'], marker='o', linestyle='-', color='b')
    plt.title('Temperatura promedio diario mensual por Año')
    plt.xlabel('Fecha')
    plt.ylabel('Temperatura Promedio (°C)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def f_matriz_correlacion_variables(datos):
  # Calcular la matriz de correlación
  corr_matrix = datos.corr()

  # Visualizar la matriz de correlación
  plt.figure(figsize=(10, 8))
  sns.heatmap(corr_matrix, annot=True, fmt=".2f")
  plt.show()

# Esta función regresa los valores de los Coeficientes
# El intercept_0 es el coeficiene de intersección y el resto
# son los 8 coeficiene de las 8 variables predictoras del 0 al 7 por ser un arreglo
def f_coeficientes_modelo (modelo):
  return modelo.intercept_, modelo.coef_[0], modelo.coef_[1], modelo.coef_[2], modelo.coef_[3], modelo.coef_[4], modelo.coef_[5], modelo.coef_[6], modelo.coef_[7]

# funci´pn para ayudar a los hiperplanso, npumero de rengloes en la matriz de visualizaciones
def f_ajustar_y_dividir(numero):
  # Verificar si el número es impar (non)
  if numero % 2 != 0:
    numero += 1  # Hacerlo par incrementándolo en 1
  return numero / 2  # Dividir el número resultante entre 2 y retornarlo

# Función que deuelve interpretaciones con base en el valor w de acuerdo al hiperplano lineal
def f_interpretar_relacion(w, nombres_variables, dependiente):
    interpretaciones = []  # Inicializar una lista vacía para almacenar las interpretaciones

    for i, coef in enumerate(w):
        if coef > 0:
            interpretacion = "La relación entre la variable independiente " + nombres_variables[i] + " y la dependiente es ascendente. Cuando aumenta " + nombres_variables[i] + ", aumenta la variable " + dependiente
        elif coef < 0:
            interpretacion = "La relación entre la variable independiente " + nombres_variables[i] + " y la dependiente es descendente. Cuando aumenta " + nombres_variables[i] + ", disminuye la variable " + dependiente
        else:
            interpretacion = "La variable independiente " + nombres_variables[i] + " no tiene una relación lineal clara con la variable "+ dependiente
        interpretaciones.append(interpretacion)  # Añadir la interpretación actual a la lista de interpretaciones

    return interpretaciones  # Devolver la lista completa de interpretaciones


def f_hiperplanos_lineal_svr(w, b, error, nombres_variables, datos_escalados):
  # Ajustar el número de filas para los gráficos
  num_variables = len(nombres_variables)
  renglones = int(f_ajustar_y_dividir(num_variables))
  columnas = 2 if num_variables > 1 else 1

  # Configura la matriz de visualizaciones para hiperplanos debe ser cuadrada
  fig, axs = plt.subplots(renglones, 2, figsize=(15, 9))
  if renglones * columnas == 1:
    axs = np.array([axs]) # Asegurarse de que axs sea iterable

  for i, nombre_var in enumerate(nombres_variables):
     # Dibujando hiperplanos
    x_values = datos_escalados[nombre_var]
    y_hyperplano = w[0,i] * x_values + b
    y_margen_superior = y_hyperplano + error
    y_margen_inferior = y_hyperplano - error

    # Encontrar el índice de la subfigura actual
    fila = i // columnas
    columna = i % columnas

    axs[fila, columna].plot(x_values, y_hyperplano, 'r', label=f'Hiperplano para {nombre_var}')
    axs[fila, columna].plot(x_values, y_margen_superior, 'b--', label='Margen superior')
    axs[fila, columna].plot(x_values, y_margen_inferior, 'b--', label='Margen inferior')
    axs[fila, columna].set_xlabel(nombre_var)
    axs[fila, columna].set_ylabel('y')
    axs[fila, columna].legend()

  # Esconder gráficos vacíos si los hay
  for j in range(i + 1, renglones * columnas):
    fila = j // columnas
    columna = j % columnas
    axs[fila, columna].axis('off')

  plt.tight_layout()
  plt.show()

# Función para trazar las predicciones del modelo SVR para cada variable independiente en una matriz de subgráficos.
def f_plot_svr_predicciones(modelo, X_train, X_test, y_test, nombres_variables):
    """
    Función para trazar las predicciones del modelo SVR para cada variable independiente en una matriz de subgráficos.

    :param modelo: modelo SVR entrenado.
    :param X_train: conjunto de entrenamiento (para calcular los promedios).
    :param X_test: conjunto de prueba.
    :param y_test: valores reales de la variable dependiente correspondientes a X_test.
    :param nombres_variables: lista de nombres de las variables independientes.


    """

      # Ajustar el número de filas para los gráficos
    num_variables = len(nombres_variables)
    renglones = int(f_ajustar_y_dividir(num_variables))
    columnas = 4 if num_variables > 1 else 1

    X_train_mean = X_train.mean().values  # Calculamos la media de X_train
    n_vars = len(nombres_variables)
    fig, axs = plt.subplots(renglones, columnas, figsize=(20, 10))  # Ajusta el tamaño según necesites
    axs = axs.flatten()  # Aplanar el array de ejes para facilitar su manejo

    for i, nombre_var in enumerate(nombres_variables):
        variable_values = X_test[nombre_var].values
        X_pred = np.tile(X_train_mean, (len(variable_values), 1))
        indice_var = nombres_variables.index(nombre_var)
        X_pred[:, indice_var] = variable_values

        # Hacemos predicciones utilizando el modelo
        y_pred = modelo.predict(X_pred)

        # Graficar en el subgráfico correspondiente
        axs[i].scatter(variable_values, y_test, color='red', label='Datos reales', alpha=0.5)
        axs[i].scatter(variable_values, y_pred, color='blue', label='Predicciones del modelo', alpha=0.5)
        axs[i].set_xlabel(nombre_var)
        axs[i].set_ylabel('Temperatura')
        axs[i].set_title(f'{nombre_var} vs. Temp')
        if i == 0:  # Solo añadimos la leyenda al primer subgráfico para evitar repetición
            axs[i].legend()

    # Asegurarse de que no haya subgráficos vacíos si el número de variables es menor que el número total de subgráficos
    for ax in axs[n_vars:]:
        fig.delaxes(ax)

    plt.tight_layout()
    plt.show()

## Cargar widgets



In [3]:
# Crear un widget de salida
visualizar = widgets.Output()


# Función para actualizar y mostrar el contenido dinámicamente en el widget de salida
def f_visualizar_datos(datos1, datos2, titulo1, titulo2):
    with visualizar:
        #global titulo1, titulo2
        #titulo1 = "Primeros 20 registros de datos"
        visualizar.clear_output(wait=True)  # Limpiar el contenido anterior
        display(HTML(f"<h2>{titulo1}</h2>"))  # Establecer el nuevo título
        # Visualizar datos
        display(datos1)

        #titulo2 = "Últimos 20 registros de datos"
        display(HTML(f"<h2>{titulo2}</h2>"))  # Establecer el nuevo título
        # Visualizar datos
        display(datos2)

## Cargar datos



In [17]:
datos = pd.read_csv("https://raw.githubusercontent.com/rpizarrog/innovacion-empresarial/main/datos/datos_rendimiento_escolar_1500.csv")

# Llamar a la función para mostrar datos de entrenamiento ordenados por índice
f_visualizar_datos(datos.head(20), datos.tail(20), "Primeros registros de Datos", "Últimos registros de Datos")

# Para mostrar el widget de salida
display(visualizar)

Output()

## Escalar datos





In [18]:
# Escalar las características con StandardScaler
# Crear instancia de StandardScaler
escalar = StandardScaler()

# Escalar las características excluyendo 'temperatura'
datos_escalados = datos.drop(columns=['rendimiento_escolar'])

datos_escalados = escalar.fit_transform(datos_escalados)

# print (datos_escalados)

# Nombres de columnas para dejar los mismos nombres que original sin fecha ni temperatura
# horas_estudio	taza_asistencia	lectura_libros	ingeso_familiar	actividad_extraescolar	motivacion
nombres_variables = ['horas_estudio', 'taza_asistencia', 'lectura_libros', 'ingeso_familiar', 'actividad_extraescolar', 'motivacion']

# Convertir de nuevo a DataFrame y asignar los nombres de las columnas
datos_escalados = pd.DataFrame(datos_escalados, columns=nombres_variables)

# Añadir la columna 'temperatura' del DataFrame original
datos_escalados['rendimiento_escolar'] = datos['rendimiento_escolar']

# Llamar a la función para mostrar datos de entrenamiento ordenados por índice
f_visualizar_datos(datos_escalados.head(20), datos_escalados.tail(20), "Primeros registros escalados", "Últimos registros escalados")

# Para mostrar el widget de salida
display(visualizar)

Output()

## Construcción de modelos

Se sacan la variales independientes y dependiente siendo estas arreglos numpy.

In [20]:
# variables independientes
independientes = datos_escalados.drop(columns=['rendimiento_escolar']).values

# Variable dependiente
dependiente = datos_escalados['rendimiento_escolar'].values

n = independientes.shape[0] # Total de registros
p = independientes.shape[1] # Total de variables

print ("Registros", n)
print ("Predictoes o variable independientes", p)

Registros 1500
Predictoes o variable independientes 6


### Validación cruzada con modelo de regresión lineal múltiple

In [21]:
conjuntos = 5

# Definir el modelo de regresión lineal múltiple
modelo_rl = LinearRegression()

# Configurar K-Fold para la validación cruzada
kfold = KFold(n_splits=conjuntos, shuffle=True, random_state=2024)

# Calcular el puntaje de validación cruzada usando MSE como la métrica de evaluación
# Nota: cross_val_score por defecto maximiza el score, por lo que para MSE, que es un "menor es mejor", se usa su negativo.
puntajes_mse = cross_val_score(modelo_rl, independientes, dependiente, cv=kfold, scoring='neg_mean_squared_error')

# Convertir los puntajes a errores cuadráticos medios positivos
mse_scores = -puntajes_mse

print(f"Errores Cuadráticos Medios (MSE) por fold: {mse_scores}")
print(f"Promedio MSE: {np.mean(mse_scores)}")
print(f"Desviación estándar MSE: {np.std(mse_scores)}")

# Ya tienes calculados los MSE scores, así que calculamos el RMSE promedio
rmse_scores = np.sqrt(mse_scores)
promedio_rmse = np.mean(rmse_scores)
print(f"Regresión lineal multiple. Promedio RMSE: {np.round(promedio_rmse, 6)}\n")

# Para R^2, usar cross_val_score nuevamente con scoring='r2'
puntajes_r2 = cross_val_score(modelo_rl, independientes, dependiente, cv=kfold, scoring='r2')
promedio_r2 = np.mean(puntajes_r2)

# Calculando el R^2 ajustado promedio
r2_ajustado = 1 - (1-promedio_r2) * ((n-1)/(n-p-1))

print(f"Promedio r squared: {promedio_r2}")
print(f"r squared Ajustado Promedio: {r2_ajustado}")

Errores Cuadráticos Medios (MSE) por fold: [8.20549878e-06 8.35736128e-06 8.08404020e-06 8.31338219e-06
 7.41639070e-06]
Promedio MSE: 8.075334628393274e-06
Desviación estándar MSE: 3.427626757223354e-07
Regresión lineal multiple. Promedio RMSE: 0.002841

Promedio r squared: 0.9999997293357545
r squared Ajustado Promedio: 0.9999997282480215


### Modelo árbol de regresión


In [23]:
conjuntos = 5

# Definir el modelo de árbol de decisión para regresión
modelo_arbol = DecisionTreeRegressor(random_state=2024, max_depth=3)

# Configurar K-Fold para la validación cruzada
kfold = KFold(n_splits=conjuntos, shuffle=True, random_state=2024)

# Calcular el puntaje de validación cruzada usando MSE como la métrica de evaluación
puntajes_mse = cross_val_score(modelo_arbol, independientes, dependiente, cv=kfold, scoring='neg_mean_squared_error')

# Convertir los puntajes a errores cuadráticos medios positivos y calcular el RMSE
mse_scores = -puntajes_mse
rmse_scores = np.sqrt(mse_scores)

print(f"RMSE por fold: {rmse_scores}")
print(f"Árbol de regresión Promedio RMSE: {np.round(np.mean(rmse_scores), 6)}")
print(f"Desviación estándar RMSE: {np.std(rmse_scores)}\n")

# Calcular R^2 como métrica de evaluación adicional
puntajes_r2 = cross_val_score(modelo_arbol, independientes, dependiente, cv=kfold, scoring='r2')

print(f"R^2 por fold: {puntajes_r2}")
print(f"Promedio R^2: {np.round(np.mean(puntajes_r2), 2)}")
print(f"Desviación estándar R^2: {np.std(puntajes_r2)}")

RMSE por fold: [3.17211406 3.25231918 3.09262155 3.18610355 3.07734026]
Árbol de regresión Promedio RMSE: 3.1561
Desviación estándar RMSE: 0.0642610376739509

R^2 por fold: [0.66647436 0.68092482 0.64462822 0.66636548 0.67357413]
Promedio R^2: 0.67
Desviación estándar R^2: 0.012136354964044773


### Random Forest



In [24]:
# Número de conjuntos para la validación cruzada
conjuntos = 5

# Definir el modelo de Random Forest para regresión
modelo_rf = RandomForestRegressor(random_state=2024, n_estimators=300, max_depth=3)

# Configurar K-Fold para la validación cruzada
kfold = KFold(n_splits=conjuntos, shuffle=True, random_state=2024)

# Calcular el puntaje de validación cruzada usando MSE como la métrica de evaluación para Random Forest
puntajes_mse_rf = cross_val_score(modelo_rf, independientes, dependiente, cv=kfold, scoring='neg_mean_squared_error')

# Convertir los puntajes a errores cuadráticos medios positivos y calcular el RMSE para Random Forest
mse_scores_rf = -puntajes_mse_rf
rmse_scores_rf = np.sqrt(mse_scores_rf)

print(f"Random Forest - RMSE por fold: {rmse_scores_rf}")
print(f"Random Forest - Promedio RMSE: {np.round(np.mean(rmse_scores_rf), 6)}")
print(f"Random Forest - Desviación estándar RMSE: {np.std(rmse_scores_rf)}\n")

# Calcular R^2 como métrica de evaluación adicional para Random Forest
puntajes_r2_rf = cross_val_score(modelo_rf, independientes, dependiente, cv=kfold, scoring='r2')

print(f"Random Forest - R^2 por fold: {puntajes_r2_rf}")
print(f"Random Forest - Promedio R^2: {np.round(np.mean(puntajes_r2_rf), 2)}")
print(f"Random Forest - Desviación estándar R^2: {np.std(puntajes_r2_rf)}")

Random Forest - RMSE por fold: [2.72945351 2.70934727 2.50473812 2.62892456 2.68544865]
Random Forest - Promedio RMSE: 2.651582
Random Forest - Desviación estándar RMSE: 0.08077652528532994

Random Forest - R^2 por fold: [0.75306478 0.77857021 0.76689369 0.77285276 0.75141939]
Random Forest - Promedio R^2: 0.76
Random Forest - Desviación estándar R^2: 0.010726767450612


### Polinomial grado 2


In [25]:
from sklearn.pipeline import Pipeline

# Configuración inicial
conjuntos = 5
grado = 2  # Grado del polinomio

# Crear una tubería (pipeline) que primero transforme las características al espacio polinomial, y luego aplique regresión lineal
modelo_pol = Pipeline([
    ('poly', PolynomialFeatures(degree=grado)),
    ('linear', LinearRegression())
])

# Configurar K-Fold para la validación cruzada
kfold = KFold(n_splits=conjuntos, shuffle=True, random_state=2024)

# Ahora la validación cruzada

# Calcular el puntaje de validación cruzada usando MSE como la métrica de evaluación
puntajes_mse = cross_val_score(modelo_pol, independientes, dependiente, cv=kfold, scoring='neg_mean_squared_error')

# Convertir los puntajes a errores cuadráticos medios positivos y calcular el RMSE
mse_scores = -puntajes_mse
rmse_scores = np.sqrt(mse_scores)

print(f"Modelo Polinomial - RMSE por fold: {rmse_scores}", "Grado ", grado)
print(f"Modelo Polinomial - Promedio RMSE: {np.round(np.mean(rmse_scores), 6)}", "Grado ", grado)
print(f"Modelo Polinomial - Desviación estándar RMSE: {np.std(rmse_scores)}\n")

# Calcular R^2 como métrica de evaluación adicional
puntajes_r2 = cross_val_score(modelo_pol, independientes, dependiente, cv=kfold, scoring='r2')

print(f"Modelo Polinomial - R^2 por fold: {puntajes_r2}", "Grado ", grado)
print(f"Modelo Polinomial - Promedio R^2: {np.mean(puntajes_r2)}")
print(f"Modelo Polinomial - Desviación estándar R^2: {np.std(puntajes_r2)}")

Modelo Polinomial - RMSE por fold: [0.00289213 0.00288488 0.00284377 0.00291227 0.00274971] Grado  2
Modelo Polinomial - Promedio RMSE: 0.002857 Grado  2
Modelo Polinomial - Desviación estándar RMSE: 5.788177840136891e-05

Modelo Polinomial - R^2 por fold: [0.99999972 0.99999975 0.9999997  0.99999972 0.99999974] Grado  2
Modelo Polinomial - Promedio R^2: 0.999999726370109
Modelo Polinomial - Desviación estándar R^2: 1.6966785277695388e-08


### MSR kernel lineal


In [26]:
# Número de conjuntos para la validación cruzada
conjuntos = 5
error = 0.1

# Definir el modelo SVR con kernel lineal
modelo_svr_lineal = SVR(kernel='linear', epsilon=error)

# Configurar K-Fold para la validación cruzada
kfold = KFold(n_splits=conjuntos, shuffle=True, random_state=2024)

# Calcular el puntaje de validación cruzada usando MSE como la métrica de evaluación
puntajes_mse = cross_val_score(modelo_svr_lineal, independientes, dependiente, cv=kfold, scoring='neg_mean_squared_error')

# Convertir los puntajes a errores cuadráticos medios positivos y calcular el RMSE
mse_scores = -puntajes_mse
rmse_scores = np.sqrt(mse_scores)

print(f"SVR lineal - RMSE por fold: {rmse_scores}")
print(f"SVR lineal - Promedio RMSE: {np.round(np.mean(rmse_scores), 6)}")
print(f"SVR lineal - Desviación estándar RMSE: {np.std(rmse_scores)}\n")

# Calcular R^2 como métrica de evaluación adicional
puntajes_r2 = cross_val_score(modelo_svr_lineal, independientes, dependiente, cv=kfold, scoring='r2')

print(f"SVR lineal - R^2 por fold: {puntajes_r2}")
print(f"SVR lineal - Promedio R^2: {np.mean(puntajes_r2)}")
print(f"SVR lineal - Desviación estándar R^2: {np.std(puntajes_r2)}")

SVR lineal - RMSE por fold: [0.05452866 0.06049584 0.05553595 0.05721747 0.05626162]
SVR lineal - Promedio RMSE: 0.056808
SVR lineal - Desviación estándar RMSE: 0.002043519733746135

SVR lineal - R^2 por fold: [0.99990144 0.9998896  0.9998854  0.9998924  0.99989089]
SVR lineal - Promedio R^2: 0.9998919483312664
SVR lineal - Desviación estándar R^2: 5.2887665415032245e-06


### MSV kernel polinomial



In [27]:
# Número de conjuntos para la validación cruzada
conjuntos = 5
grado = 3
error = 0.1

# Definir el modelo SVR con kernel polinomial de grado 3
modelo_svr_poly = SVR(kernel='poly', degree=grado, epsilon=error)

# Configurar K-Fold para la validación cruzada
kfold = KFold(n_splits=conjuntos, shuffle=True, random_state=2024)

# Calcular el puntaje de validación cruzada usando MSE como la métrica de evaluación
puntajes_mse = cross_val_score(modelo_svr_poly, independientes, dependiente, cv=kfold, scoring='neg_mean_squared_error')

# Convertir los puntajes a errores cuadráticos medios positivos y calcular el RMSE
mse_scores = -puntajes_mse
rmse_scores = np.sqrt(mse_scores)

print(f"SVR Polinomial - RMSE por fold: {rmse_scores}")
print(f"SVR Polinomial - Promedio RMSE: {np.round(np.mean(rmse_scores), 6)}")
print(f"SVR Polinomial - Desviación estándar RMSE: {np.std(rmse_scores)}\n")

# Calcular R^2 como métrica de evaluación adicional
puntajes_r2 = cross_val_score(modelo_svr_poly, independientes, dependiente, cv=kfold, scoring='r2')

print(f"SVR Polinomial - R^2 por fold: {puntajes_r2}")
print(f"SVR Polinomial - Promedio R^2: {np.mean(puntajes_r2)}")
print(f"SVR Polinomial - Desviación estándar R^2: {np.std(puntajes_r2)}")

SVR Polinomial - RMSE por fold: [0.64714165 0.62111395 0.57276912 0.56481156 0.57640278]
SVR Polinomial - Promedio RMSE: 0.596448
SVR Polinomial - Desviación estándar RMSE: 0.032067491341785305

SVR Polinomial - R^2 por fold: [0.98611869 0.98836278 0.98781042 0.98951524 0.98854789]
SVR Polinomial - Promedio R^2: 0.9880710052117045
SVR Polinomial - Desviación estándar R^2: 0.0011205171756110573


### MSV kernel radial



In [28]:
# Número de conjuntos para la validación cruzada
conjuntos = 5
error = 0.1

# Definir el modelo SVR con kernel Radial de grado 3
modelo_svr_radial = SVR(kernel='rbf', epsilon=error)

# Configurar K-Fold para la validación cruzada
kfold = KFold(n_splits=conjuntos, shuffle=True, random_state=2024)

# Calcular el puntaje de validación cruzada usando MSE como la métrica de evaluación
puntajes_mse = cross_val_score(modelo_svr_radial, independientes, dependiente, cv=kfold, scoring='neg_mean_squared_error')

# Convertir los puntajes a errores cuadráticos medios positivos y calcular el RMSE
mse_scores = -puntajes_mse
rmse_scores = np.sqrt(mse_scores)

print(f"SVR Radial - RMSE por fold: {rmse_scores}")
print(f"SVR Radial - Promedio RMSE: {np.mean(rmse_scores)}")
print(f"SVR Radial - Desviación estándar RMSE: {np.round(np.std(rmse_scores), 6)}\n")

# Calcular R^2 como métrica de evaluación adicional
puntajes_r2 = cross_val_score(modelo_svr_radial, independientes, dependiente, cv=kfold, scoring='r2')

print(f"SVR Radial - R^2 por fold: {puntajes_r2}")
print(f"SVR Radial - Promedio R^2: {np.mean(puntajes_r2)}")
print(f"SVR Radial - Desviación estándar R^2: {np.std(puntajes_r2)}")

SVR Radial - RMSE por fold: [0.45155252 0.5236019  0.38103764 0.47262859 0.48680934]
SVR Radial - Promedio RMSE: 0.4631259997142795
SVR Radial - Desviación estándar RMSE: 0.047288

SVR Radial - R^2 por fold: [0.99324153 0.99172993 0.99460533 0.99265839 0.99183133]
SVR Radial - Promedio R^2: 0.9928133026356953
SVR Radial - Desviación estándar R^2: 0.00105420045436528


## ¿Mejor modelo?

¿Cuál fue el mejor modelo de regresión con estos datos de acuerdo al estadístico *RMSE*?

* Regresión lineal múltiple. Promedio RMSE: 0.002841
* Árbol de regresión Promedio RMSE: 3.1561
* Random Forest - Promedio RMSE: 2.651582
* Modelo Polinomial - Promedio RMSE: 0.002857 Grado  2
* Modelo Polinomial - Promedio RMSE: 0.002921 Grado  3
* Modelo Polinomial - Promedio RMSE: 0.003077 Grado  4
* SVR lineal - Promedio RMSE: 0.056808
* SVR Polinomial - Promedio RMSE: 0.596448
* SVR Radial - Promedio RMSE:  0.047288

De acuerdo al estadístico *RMSE* el mejor modelo de regresión para estos datos fue: **Regresión lineal múltiple. Promedio RMSE: 0.002841**

Todos los modelos tienen un valor mayor al 95% en *r squared* que significan que las variables independientes explican por encima del 95% la variable objetivo rendimiento escolar.


## Predecir un nuevo dato

Si el modelo ha sido evaluado usando validación cruzada pero aún no ha sido entrenado con todos los datos, hay que entrenar el modelo.

La validación cruzada es una técnica para evaluar cómo el modelo generalizará a un conjunto de datos independiente, pero después de esta evaluación, típicamente se entrenará el modelo en todos los datos disponibles antes de usarlo para hacer predicciones.

¿Cuál será la prediccón de un nuevo registro con estos valores en sus variables independientes?:

* horas_estudio	: 2
* taza_asistencia: 86
* lectura_libros	: 2
* ingeso_familiar	: 60000
* actividad_extraescolar	: 3
* motivacion: 0




In [14]:
# Los valores en las variables
v_horas_estudio	= 2
v_taza_asistencia= 86
v_lectura_libros	= 2
v_ingreso_familiar	= 60000
v_actividad_extraescolar	= 3
v_motivacion= 0

nuevo_registro = np.array([[v_horas_estudio,
                   v_taza_asistencia,
                   v_lectura_libros,
                   v_ingreso_familiar,
                   v_actividad_extraescolar,
                   v_motivacion]])

# horas_estudio	taza_asistencia	lectura_libros	ingeso_familiar	actividad_extraescolar	motivacion
nombres_variables = ['horas_estudio', 'taza_asistencia', 'lectura_libros', 'ingeso_familiar', 'actividad_extraescolar', 'motivacion']

# Crear un DataFrame con los nombres de las columnas adecuados
nuevo_registro_df = pd.DataFrame(nuevo_registro, columns=nombres_variables)

nuevo_registro_escalado = escalar.transform(nuevo_registro_df)

nuevo_registro_escalado = pd.DataFrame(nuevo_registro_escalado, columns=nombres_variables)

print("Nuevo registro escalado: ")
print(nuevo_registro_escalado)


Nuevo registro escalado: 
   horas_estudio  taza_asistencia  lectura_libros  ingeso_familiar  \
0      -0.641841        -0.651284       -1.284264         0.211033   

   actividad_extraescolar  motivacion  
0                0.717268   -1.001334  


## Entrenar el modelo

Entrenar el modelo de regesión múltiple



In [15]:
# Preparar los datos para el modelo
X = datos_escalados.drop(columns=['rendimiento_escolar'])  # Solo dejar las variables independientes o predictoras
y = datos_escalados['rendimiento_escolar']  # Variable dependiente

# Dividir los datos en conjuntos de entrenamiento y prueba o validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024)


modelo_rl.fit(X_train, y_train)

LinearRegression()

## Predicción

Hacer predicciones con el modelo entrenado

In [16]:
# Hacer la predicción con el modelo SVR
# Asumiendo que 'modelo' es el modelo SVR entrenado
prediccion_rendimiento_escolar = modelo_rl.predict(nuevo_registro_escalado)
print ("La prediccón de rendimiento escolar será de: ",prediccion_rendimiento_escolar)

La prediccón de rendimiento escolar será de:  [78.12357743]


# Interpretación del caso

Los datos que se utilizaron en este caso fue de variables independientes  que afectan al rendimiento escolar de un estudiante.

Las variables independiente fueron:
* Horas de estudio por semana,
* Asistencia a clases (tasa de asistencia),
* Número de libros leídos por año,
* Ingreso familiar,
* Número de actividades extracurriculares y
* Nivel de motivación.

La variable dependiente es un valor numérico de rendimiento escolar.

¿Para que sirve esclar datos?, para homogeneizar valores en una misa escala de tal forma ue ayuda a la eficiencia de modelos de regresión sobre todo en los de máquinas de soprote de regresión SVR. Para este caso se escalaron los datos y todos los modelos utilizaron estos datos escalados.

¿Para que sirve la técnica de validación cruzada en modelos de regresión?

Para evaluar modelo de manera generalizada y permite con ello observar eficiencia de modelos para luego entrenar el mejor modelo y con ello realizar predicciones.

¿Cuál fue el mejor modelo?. **Regresión lineal múltiple.**

Las porcetaje explican las variables independientes a la variable dependiene en el modelo de regresión lineal múltiple? Por encima del 95% de acuerdo al valor de *r squared* y *r suquared ajustado*

¿Que predicción tiene el modelo de regresión lineal múltiple para un nuevo registro de un estudiante con valores de: horas_estudio	: 2; taza_asistencia: 86; lectura_libros	: 2; ingeso_familiar	: 60000; actividad_extraescolar	: 3; motivacion: 0. La prediccón del rendimiento escolar es de: **78.12**

